In [1]:
import glob
import numpy as np
import rasterio
from rasterio.plot import show
from lab5functions import slopeAspect, reclassAspect

In [2]:
IN_DATA_DIR = r'C:\Users\delaney\Desktop\geog5092_workspace\final_project\in_data'
OUT_DATA_DIR = r'C:\Users\delaney\Desktop\geog5092_workspace\final_project\out_data'

master_crs = 'epsg:26955'

tifs = sorted(glob.glob(IN_DATA_DIR + r'\*.tif'))

In [3]:
with rasterio.open(tifs[0]) as DEM:
    DEM_array = DEM.read(1)
    slp, aspect = slopeAspect(DEM_array, 30)
    reclass_aspect = reclassAspect(aspect)
    slope_suit = np.where((slp > 0) & (slp <= 15), 0,
             np.where((slp > 15) & (slp <= 30), 1,
             np.where((slp > 30) & (slp <= 45), 2,
             np.where((slp > 45) & (slp <= 60), 3,
             np.where((slp > 60) & (slp <= 90), 4, 0)))))
    aspect_suit = np.where((reclass_aspect == 4) | (reclass_aspect == 5) | (reclass_aspect == 6), 1, 0)

with rasterio.open(tifs[1]) as roads:
    roads_array = roads.read(1)
    roads_suit = np.where(roads_array == 1, 0, 1)

with rasterio.open(tifs[2]) as trails:
    trails_array = trails.read(1)
    trails_suit = np.where(trails_array == 1, 0, 1)

C:\Users\delaney\Desktop\geog5092_workspace\final_project\lab5functions.py:31: RuntimeWarning: overflow encountered in square
  slp = np.arctan((dzdx ** 2 + dzdy ** 2) ** 0.5) * 180 / pi


In [4]:
full_sum = roads_suit + trails_suit + slope_suit + aspect_suit
top_sites = np.where(full_sum >= 5, 1, 0)

In [5]:
top_unique, top_counts = np.unique(top_sites, return_counts = True)

n = 900
top_area = (np.multiply(top_counts[1], n))/1000000

print('Total area of the top nesting sites:', round(top_area, 2), 'sq. km.')

Total area of the top nesting sites: 17.06 sq. km.


In [6]:
template = rasterio.open(tifs[0])

with rasterio.open(OUT_DATA_DIR + r'\top_sites.tif', 'w',
                   driver = 'GTiff',
                   height = top_sites.shape[0],
                   width = top_sites.shape[1],
                   count = 1,
                   dtype = 'int8',
                   transform = template.transform,
                   crs = master_crs,
                   nodata = 0
) as out_raster:
    top_sites = top_sites.astype('int8')
    out_raster.write(top_sites, indexes = 1)
    
print("Export 1 complete.")


with rasterio.open(OUT_DATA_DIR + r'\full_sum.tif', 'w',
                   driver = 'GTiff',
                   height = full_sum.shape[0],
                   width = full_sum.shape[1],
                   count = 1,
                   dtype = 'int8',
                   transform = template.transform,
                   crs = master_crs,
                   nodata = 0
) as out_raster:
    full_sum = full_sum.astype('int8')
    out_raster.write(full_sum, indexes = 1)

print("Export 2 complete.")

Export 1 complete.
Export 2 complete.
